In [13]:
import pandas as pd
import glob
import re

results = glob.glob('test_run/vgg19aw_ucb/**/progress.csv')
tems = []
accs = []


for result in results:
    f = pd.read_csv(result)

    t = result.split('/')[-2]
    tem = re.search(r't0=\S+t1=\S+t2=\S+t3=\S+t4=\S{5}', t).group()
    tems.append(tem)
    accs.append(max(f['accuracy']))

df = pd.DataFrame({'temperature': tems, 'accuracy': accs})

df = df.sort_values(by=['accuracy'], ascending=False)



    


,temperature,accuracy
6,"t0=0.0413,t1=0.9038,t2=3.3750,t3=7.9981,t4=7.984",0.5546
29,"t0=0.0819,t1=0.1011,t2=1.9517,t3=9.2978,t4=8.361",0.5544
2,"t0=0.4042,t1=0.3046,t2=1.3907,t3=7.4739,t4=5.521",0.5520
1,"t0=0.2183,t1=0.1511,t2=3.3127,t3=7.7989,t4=3.086",0.5512
24,"t0=0.1357,t1=0.6517,t2=1.2487,t3=7.9768,t4=9.674",0.5502
17,"t0=0.3059,t1=0.1395,t2=2.1686,t3=4.9309,t4=5.648",0.5502
22,"t0=0.4038,t1=0.1758,t2=2.2669,t3=4.1509,t4=7.807",0.5500
11,"t0=0.3641,t1=0.6675,t2=4.9781,t3=5.2527,t4=6.309",0.5500
27,"t0=0.0610,t1=0.4952,t2=1.1376,t3=9.2746,t4=4.070",0.5496
14,"t0=0.3124,t1=0.7630,t2=2.5341,t3=3.5196,t4=8.700",0.5494
